In [1]:
from skimage.io import collection, imread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import re
import glob

from datetime import datetime

# How is the public opinion about a company correlated to it's market value?

A company's market value is variable and depends on a lot of factors. The price is a reflection of the company's perceived value – what the public is willing to pay for a piece of the company. It can and will rise and fall, based on a variety of factors in the global landscape and within the company itself. One of which is becoming more influential than ever - the public's opinion on social media.

To analyze this correlation we'll look at two datasets. The first one contains over 3 million unique tweets with their information such as tweet id, author of the tweet, post date, the text body of the tweet, and the number of comments, likes, and retweets of tweets matched with the related company.

The second one will just have daily stock price records (from the Forbes2000) for us to make a reference with.

### 1. Data Acquisition 

So first let's read the tweets dataset into pandas and inspect a small sample from the two dataframes.

In [2]:
tweets = pd.read_csv('./top-companies-tweets/Tweet.csv')
tweets.sample(5)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
3069201,1096509337198051328,MC_OptionTrades,1550263172,$TSLA 20-Sep-19 375 Calls Traded 530 times for...,0,0,0
77508,565264725051981826,treabase,1423604587,"[Apple Inc] #patent 8,951,071: Contact-support...",0,0,0
1814037,863860544507346944,Milennial_Bull,1494795378,"Already loaded on lithium ETFs, suspect it mig...",0,0,0
797939,695768656228167680,PolgarStocks,1454719149,"$AG News Updated Friday, February 5, 2016 6:59...",0,0,0
1562599,813780125372350466,daytrend,1482855275,Anyone with IoT creeping into their home shoul...,0,0,2


In [3]:
tweets_company = pd.read_csv('./top-companies-tweets/Company_Tweet.csv')
tweets_company.sample(5)

,tweet_id,ticker_symbol
1472120,601698805944188929,GOOG
1784452,1107567314852028416,GOOG
2747274,1100834023465734144,AMZN
1377911,1176509533742673920,AAPL
3107477,924649853082394624,MSFT


So next up let's read the datasets for each of the stocks, which we are monitoring. We will save them in a dictionary with the key, being the company's tick name and the value - it's stock prices over time dataset.

In [4]:
stocks_df = {}
for name in glob.glob('./stocks/*'):
    stocks_df[name.split('\\')[-1].split('.')[0]] = pd.read_csv(name)
stocks = pd.concat(stocks_df)
stocks.sample(10)

Date         Low        Open     Volume        High  \
AAPL 7533  21-10-2010   10.957143   11.155714  551460000   11.240714   
MSFT 4262  31-01-2003   23.514999   23.725000  106858400   24.174999   
AMZN 2143  18-11-2005   46.660000   46.900002   59332800   48.000000   
AAPL 90    23-04-1981    0.130580    0.130580   58016000    0.131138   
     6014  11-10-2004    0.682143    0.692857  323870400    0.697500   
GOOG 282   30-09-2005  155.561920  156.523315   18371203  158.157181   
AMZN 1235  16-04-2002   13.720000   14.180000    3681000   14.290000   
MSFT 1348  12-07-1991    1.375000    1.395833   35174400    1.427083   
AAPL 3316  24-01-1994    0.296875    0.296875  692148800    0.314732   
     5036  14-11-2000    0.349330    0.356027  409001600    0.366071   

                Close  Adjusted Close  
AAPL 7533   11.054286        9.505616  
MSFT 4262   23.730000       15.037395  
AMZN 2143   47.980000       47.980000  
AAPL 90      0.130580        0.102503  
     6014    0.689107        0.592565  
GOOG 282   157.639130      157.639130  
AMZN 1235   14.010000       14.010000  
MSFT 1348    1.416667        0.897724  
AAPL 3316    0.312500        0.261996  
     5036    0.361607        0.310947

### 2. Data Tidying and Cleaning

First let's combine the two tables from the twitter dataset and convert the dates to a datetime object.

In [5]:
tweets = pd.merge(tweets, tweets_company, on = "tweet_id")
tweets.sample(5)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol
4147195,1181666941922004992,mnguyen_saa,1570566327,"VW Posts Weak September US Car Sales, Bets Big...",0,0,0,TSLA
946890,699951728549036032,JoaquimAgusti,1455716471,Amazon And Microsoft In The Cloud: How They're...,0,0,0,AMZN
461837,620527205500780545,PennyStockingt,1436780189,Our Stock Pick On $GEQU Nearly QUADRUPLED In V...,0,0,0,MSFT
341647,600825290030321664,Benzinga,1432082886,Apple's Tim Cook Calls For Civil Rights At Com...,0,3,1,AAPL
530910,628335524554653696,JohnyTradr,1438641837,"Vanguard STAR Fund, August 2015 http://snip.ly...",0,0,0,AAPL


We will see what timeframe does our dataset cover, by getting the data of the earliest and latest tweets.

In [6]:
tweets.post_date = pd.to_datetime(tweets.post_date, unit='s')
tweets.post_date.min(), tweets.post_date.max()

(Timestamp('2015-01-01 00:00:57'), Timestamp('2019-12-31 23:55:53'))

So it has data from 01.01.2015 to 31.12.2019, so basically from 2015 to the beginning of 2020. Knowing this we can filter out the stock prices to be only in this period of time. But first we have to covert the "Date" column to datetime.

In [7]:
def string_to_date(date_string):
    return datetime.strptime(date_string, "%d-%m-%Y")
stocks.Date = stocks.Date.apply(string_to_date)

In [8]:
stocks = stocks[(stocks.Date >= '01-01-2015') & (stocks.Date < '01-01-2020')]
stocks.sample(10)

Date         Low        Open    Volume        High  \
AMZN 4748 2016-03-30  595.000000  596.710022   3890500  603.239990   
TSLA 2277 2019-07-17   50.669998   51.133999  48823500   51.661999   
MSFT 7522 2016-01-13   51.299999   53.799999  66883600   54.070000   
     7415 2015-08-11   45.900002   46.820000  29237400   46.939999   
     7588 2016-04-19   55.680000   56.630001  29596800   56.770000   
     8434 2019-08-28  133.550003  134.880005  17393300  135.759995   
GOOG 2814 2015-10-22  644.010010  646.700012   4071000  657.799988   
MSFT 8125 2018-06-06  101.900002  102.480003  21122900  102.599998   
GOOG 3314 2017-10-17  988.590027  990.289978   1290200  996.440002   
MSFT 7417 2015-08-13   46.490002   47.060001  22627200   47.099998   

                Close  Adjusted Close  
AMZN 4748  598.690002      598.690002  
TSLA 2277   50.972000       50.972000  
MSFT 7522   51.639999       46.906551  
     7415   46.410000       41.599380  
     7588   56.389999       51.588913  
     8434  135.559998      133.069611  
GOOG 2814  651.789978      651.789978  
MSFT 8125  102.489998       98.670975  
GOOG 3314  992.179993      992.179993  
MSFT 7417   46.730000       41.886223

As we can see we don't have data for every day, because the stock market funcitons only on workdays, unlike twitter.  We will find a way to work around this later.

Now lets strip the data down to just one column - the value, which we will calculate by getting the mean of the Open and Close prices. 

In [9]:
stocks["Value"] = (stocks.Open + stocks.Close) / 2
stocks = stocks.drop(columns=['Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close'])
stocks.sample(10)

Date        Value
MSFT 8054 2018-02-23    93.829998
     8387 2019-06-21   136.775002
AAPL 9602 2019-01-11    38.146250
MSFT 8176 2018-08-17   107.470001
GOOG 2772 2015-08-24   581.304993
     3275 2017-08-22   918.704987
AMZN 5109 2017-09-01   981.225006
GOOG 3332 2017-11-10  1027.264954
MSFT 7841 2017-04-20    65.480000
AAPL 8607 2015-01-29    29.402500

 So next up lets take a look at the datatpes and null values for the twitter dataset.

In [14]:
tweets.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4336445 entries, 0 to 4336444
Data columns (total 8 columns):
tweet_id         4336445 non-null int64
writer           4280526 non-null object
post_date        4336445 non-null datetime64[ns]
body             4336445 non-null object
comment_num      4336445 non-null int64
retweet_num      4336445 non-null int64
like_num         4336445 non-null int64
ticker_symbol    4336445 non-null object
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 297.8+ MB
